In [13]:
ntrees = 15

In [12]:
from tqdm import tqdm
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.naive_bayes import BernoulliNB ## check
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing, metrics
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [54]:
from tensorflow.python.framework import ops

In [56]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [22]:
def GetTreeSplits(rf):
    forest = rf.estimators_
    featurelist=[]  #collection of arrays, one for each tree
    threshlist=[]
    trees = []

    for i in range(len(forest)):
        trees.append(forest[i].tree_)
        featurelist.append(np.asarray(trees[i].feature))
        threshlist.append(np.asarray(trees[i].threshold))

    return (trees, featurelist, threshlist)


def GetChildren(trees):
    listcl = []
    listcr = []
    for i in range(len(trees)):
        listcl.append(trees[i].children_left)
        listcr.append(trees[i].children_right)
    return(listcl, listcr)



def GetActiveNodes(sample,featurelist, threshlist, trees):
    #returns a list over all trees with all corresponding active nodes
    #(featurelist, threshlist, trees) = GetTreeSplits(rf)

    act_node_list=[]      #list, containing all reached nodes for each tree listed

    for i in range(len(featurelist)):
        actives = []
        currentnode = 0
        actives.append(0)

        while featurelist[i][currentnode] != -2:    #not at end leaf yet

            if sample[featurelist[i][currentnode]] >= threshlist[i][currentnode]:
                currentnode = trees[i].children_right[currentnode]
            else:
                currentnode = trees[i].children_left[currentnode]

            actives.append(currentnode)

        act_node_list.append(actives)

    return act_node_list

def node_indicator_function(sample, rf):
    #returns a vector of node activities over all tree nodes, over all trees.
    #The output is
    #zero for inactive nodes, and 1 for active nodes. In case of the queried
    #node being a leaf node, the predicted class is returned as well.

    (featurelist, threshlist) = GetTreeSplits(rf)
    (act_node_list, active_feats_list, active_threshs_list) = GetActiveNodes(sample, rf)

    label = 0
    leafind = False
    if act_node_list[treei].count(nodej) > 0:   #if active
        indicator = 1.0
        #print treei, nodej, len(act_node_list[treei])
        if act_node_list[treei][len(act_node_list[treei])-1]==nodej: #if end leaf
            label = rf.predict(sample)
            leafind = True

    else:
        indicator = 0.0

    return (indicator, label, leafind)

In [23]:
# import numpy as np
# from forest_functions import GetTreeSplits, GetChildren


def InitFirstLayer(rf, strength01 = 1000.0):
    """
    Given a fitted random regression forest model rf, this function returns
    initialisation parameters for the first hidden layer of a neural net that
    mimics the random forest's prediction behaviour.
    - strength01 is a hyperparameter that determines how strongly the discrete
    step function is approximated.
    """

    # extract tree parameters
    trees, featurelist, threshlist = GetTreeSplits(rf)
    listcl, listcr = GetChildren(trees)

    # layer sizes
    HL1N = sum( [np.sum(tree.feature != -2) for tree in trees] )
    n_inputs = rf.n_features_

    # initialise first layer parameters
    W1 = np.zeros( [n_inputs, HL1N], dtype = 'float32')
    b1 = np.zeros([HL1N])
    b1 = np.array(b1, dtype = 'float32')


    currentnode = 0	# index of HL1 neuron to be assigned an input weight
    nodelist = []	# list (over all trees) of list of network neurons
                      # corresponding to tree splits

    # for every tree
    for i in range(len(trees)):
        cl = listcl[i]
        cr = listcr[i]

        currentsplit = 0  # index for current node while moving through tree i
        nodeCount = trees[i].node_count   # number of nodes in tree i
        nlist = np.zeros([nodeCount, 1])  # for each treenode save a neuronindex

        # go through all tree nodes in tree i
        while currentsplit < nodeCount:
            """
            Interpretation help:
            #active with (+1) if Wx>b 	<==> active (+1) when split to the right
            #active with (-1) if Wx<=b	<==> active (-1) when split to the left.
            """

            # What to do here: set weight and bias for current HL1 neuron
            if featurelist[i][currentsplit] != -2:   #not leaf node
                W1[featurelist[i][currentsplit], currentnode] = 1.0 * strength01
                b1[currentnode] = -threshlist[i][currentsplit] * strength01

                # store relationship from tree split index to HL1 neuron index
                nlist[currentsplit] = currentnode
                currentnode += 1

            # Where to go next: identify next node in tree (depth first)
            if cl[currentsplit] != -1: #( -1 means: empty child)
                currentsplit = cl[currentsplit]
            elif cr[currentsplit] != -1 :
                currentsplit = cr[currentsplit]
            else :
                currentsplit += 1
                #move to parent and next branch

        nodelist.append(nlist) #saving list of neurons corresponding to tree i.

    return W1, b1, nodelist




def GetTreePaths(trees):
    # List of lists containing the node indices for all paths through all trees
    jointsplitindlists = []
    # Litt of lists containing all path orders (left/right) through all trees
    jointsplitorderlists = []

    # lists of left and right children
    listcl, listcr = GetChildren(trees)

    for i in range(len(trees)):
        paths, orders = [], []
        cl = listcl[i].tolist()
        cr = listcr[i]

        leaf_nodes = np.where(cr == -1)[0].tolist()
        cr = cr.tolist()

        # for every leaf node get the path that led to it.
        for leaf in leaf_nodes:
            path, order = [], []
            c = leaf
            while c != 0:
                #find mother node of c
                if c in cl:
                    mother = cl.index(c)
                    direction = -1
                else:
                    mother = cr.index(c)
                    direction = +1
                c = mother
                path.append(c)
                order.append(direction)

            path.reverse()
            order.reverse()
            paths.append(path)
            orders.append(order)

        jointsplitindlists.append(paths)
        jointsplitorderlists.append(orders)

    return jointsplitindlists, jointsplitorderlists




#nodelist is from HL1
def InitSecondLayer(rf, nodelist, strength12=0.1,  L2param=0.8):
    """
    Given a fitted random regression forest model rf,
    a nodelist from the previous layer initialisation and hyperparameters,
    this function returns initialisation parameters for the second hidden layer
    of a neural net that mimics the random forest's prediction behaviour.
    """

    # extract tree paths
    trees = [rf.estimators_[i].tree_ for i in range(rf.n_estimators) ]
    jointsplitindlists, jointsplitorderlists = GetTreePaths(trees)

    # layer sizes
    HL1N = sum( [np.sum(tree.feature != -2) for tree in trees] )
    HL2N = sum( [np.sum(tree.feature == -2) for tree in trees] )

    # empty weight matrix and bias vector
    W2 = np.zeros([HL1N,HL2N], dtype = 'float32')
    b2 = np.zeros(HL2N, dtype = 'float32')

    fneurons = []   #the feature neuron in HL1 that occurs in a given path
    dneurons = []   #the path directions of the fneurons of a given path.
    leaf_neurons = [] #for storing indices of HL2-neurons belonging to leafs
    counter = 0

    # for each tree i
    for i in range(len(trees)):
        #get relevant split info
        indlist = jointsplitindlists[i]
        orderlist = jointsplitorderlists[i]
        neurons_used = []

        # identify split neurons in HL1 and their desired direction
        for k in range (len(indlist)):
            fneurons.append( nodelist[i] [ indlist[k] ] )
            dneurons.append( orderlist[k] )
            neurons_used.append(counter)
            counter +=1

        #append HL2-neuron-indices used in tree i.
        leaf_neurons.append(neurons_used)

        # set input weights and biases for second layer
        scndlayercount = 0
        for k in range(len(fneurons)): # for every feature neuron used (in HL1)
            inputns = fneurons[k]
            dirs = dneurons[k]
            for j in range(len(dirs)):
                W2[int(inputns[j]), scndlayercount] = dirs[j] * strength12
                b2[scndlayercount] = (-len(dirs)+0.5) * strength12
            scndlayercount +=1

    return W2, b2, leaf_neurons



def InitThirdLayer(rf, leaf_neurons):
    """
    Given a fitted random regression forest model rf,
    and a list of leaf_neurons from the previous layer initialisation,
    this function returns initialisation parameters for the third output layer
    of a neural net that mimics the random forest's prediction behaviour.
    """


    trees = [rf.estimators_[i].tree_ for i in range(rf.n_estimators) ]
    ntrees = rf.n_estimators

    # layer size
    HL2N = sum( [np.sum(tree.feature == -2) for tree in trees] )

    # empty weight vector. No bias, simply linear transformation.
    W3 = np.zeros([HL2N, 1], dtype = 'float32')

    # loop over trees
    for i in range(len(trees)):
        # identify the leaf node indices for tree i
        leaf_ind = np.where(trees[i].feature ==-2)

        # get the regression value for each of those leaves
        leaf_values = [e[0][0] for e in trees[i].value[leaf_ind].tolist()]

        # HL2 neurons corresponding to tree i
        tree_neurons = leaf_neurons[i]

        #compute  weights to output layer
        for k in range(len(leaf_values)):
            W3[tree_neurons[k]] = leaf_values[k] / float(ntrees) * 0.5

    return W3

In [24]:
def get_network_initialisation_parameters(rf, strength01=100.0, strength12=1.0):
    """
    Given a pre-trained random forest model, this function returns as numpy arrays
    the weights and biases for initialising a 2-layer feedforward neural network.
    The strength01 and strength12 are hyperparameters that determine how strongly
    the continuous neural network nonlinearity will approximate a discrete step function
    """

    # get network parameters for first hidden layer
    W1, b1, nodelist1 = InitFirstLayer(rf, strength01)

    # get network parameters for second hidden layer
    W2, b2, leaf_neurons = InitSecondLayer(rf, nodelist1, strength12)

    # get network parameters for third hidden layer
    W3 = InitThirdLayer(rf, leaf_neurons)

    return W1, b1, W2, b2, W3

In [77]:

def define_forward_pass(X, init_parameters, n_inputs, HL1N, HL2N, sigma=1.0,
                        keep_sparse=True, n_layers=2):
    """
    Defines a Multilayer Perceptron (MLP) model.
    Inputs:
        - X: tf placeholder [batchsize, n_inputs] for feature inputs
        - init_parameters: tuple of numpy arrays (or None), holding values to initialise
                the network weight matrices (and biases). If None, random values are
                used for network initialisation.
        - n_inputs: number of features
        - HL1N: hidden layer 1 size
        - Hl2N: hidden layer 2 size
        - sigma: Variance of Gaussian distribution used when randomly initalising weights
        - keep_sparse: whether to enforce network weight sparsity
        - n_layers: number of hidden layers. Default case is 2, but can also run with 1 or 3,
            but the number of neurons is then defined by HL1N, HL2N.
            Note: The RF initialisation can only be used with n_layers = 2.
    Outputs:
        - predictions: tf tensor holding model predictions
    """

    if n_layers == 2:   # default case.

        # set the inital network parameter values, either to random, ...
        if init_parameters is None:
            # random initial parameters
            W_01 = tf.Variable(tf.random_normal([n_inputs, HL1N], 0.0, sigma))
            W_12 = tf.Variable(tf.random_normal([HL1N, HL2N], 0.0, sigma))
            W_23 = tf.Variable(tf.random_normal([HL2N, 1], 0.0, sigma))
            b_1 = tf.Variable(tf.random_normal([HL1N], 0.0, sigma))
            b_2 = tf.Variable(tf.random_normal([HL2N], 0.0, sigma))
            b_3 = tf.Variable(tf.random_normal([1], 0.0, sigma))

        else:
            # ... or to the specific values induced by the Random-Forest
            W1, b1, W2, b2, W3 = init_parameters

            W_01 = tf.Variable(W1)
            W_12 = tf.Variable(W2)
            W_23 = tf.Variable(W3)
            b_1 = tf.Variable(b1)
            b_2 = tf.Variable(b2)
            b_3 = tf.Variable( np.sum(W3) )

            if keep_sparse:
                mask1 = tf.constant(np.float32(W1!=0.0))
                mask2 = tf.constant(np.float32(W2!=0.0))
                W_01 = tf.multiply(mask1, W_01)
                W_12 = tf.multiply(mask2, W_12)

        # defining the network with the given weights/biases
        h = tf.nn.tanh(tf.matmul(X, W_01) + b_1)
        h2 = tf.nn.tanh(tf.matmul(h, W_12) + b_2 )
        prediction = tf.matmul(h2, W_23) + b_3


    elif n_layers == 1:   # standard 1-layer MLP, initialised randomly
        W_01 = tf.Variable(tf.random_normal([n_inputs, HL1N], 0.0, sigma))
        W_12 = tf.Variable(tf.random_normal([HL1N, 1], 0.0, sigma))
        b_1 = tf.Variable(tf.random_normal([HL1N], 0.0, sigma))
        b_2 = tf.Variable(tf.random_normal([1], 0.0, sigma))
        h = tf.nn.tanh(tf.matmul(X, W_01) + b_1)
        prediction = tf.matmul(h, W_12) + b_2

    elif n_layers == 3:   # standard 3-layer MLP, initialised randomly
        W_01 = tf.Variable(tf.random_normal([n_inputs, HL1N], 0.0, sigma))
        W_12 = tf.Variable(tf.random_normal([HL1N, HL2N], 0.0, sigma))
        W_23 = tf.Variable(tf.random_normal([HL2N, HL2N], 0.0, sigma))
        W_34 = tf.Variable(tf.random_normal([HL2N, 1], 0.0, sigma))
        b_1 = tf.Variable(tf.random_normal([HL1N], 0.0, sigma))
        b_2 = tf.Variable(tf.random_normal([HL2N], 0.0, sigma))
        b_3 = tf.Variable(tf.random_normal([HL2N], 0.0, sigma))
        b_4 = tf.Variable(tf.random_normal([1], 0.0, sigma))

        h = tf.nn.tanh(tf.matmul(X, W_01) + b_1)
        h2 = tf.nn.tanh(tf.matmul(h, W_12) + b_2 )
        h3 = tf.nn.tanh(tf.matmul(h2, W_23) + b_3 )
        prediction = tf.matmul(h3, W_34) + b_4

    return prediction




def run_neural_net(XTrain, XTest, YTrain, YTest, init_parameters=None, HL1N=20, HL2N=10, n_layers=2,
                   learning_rate=0.001, forest=None, keep_sparse=True,
                   batchsize=32, n_iterations=100):
    """
    Trains / evaluates a Multilayer perceptron (MLP), potentially with a prespecified
    weight matrix initialisation.
    Inputs:
    - data: tuple of input (X) - output (Y) data for train/dev/test set.
        Output of dataloader.split_data
    - init_parameters: output of initialiser.get_network_initialisation_parameters.
        if init_parameters is set to None, random initial weights are picked.
    - HL1N: number of neurons in first hidden layer
    - HL2N: number of neurons in second hidden layer
    - n_layers: number of hidden layers. Default 2, but can also be used with 1 and 3.
    - verbose: how much to print
    - learning_rate: used during training
    - forest: a pre-trained random forest model. Not relevant when random initialisation is used.
    - keep_sparse: whether to enforce weight matrix sparsity during training
    - batchsize: used during training
    - n_iterations: Number of training epochs
    """

    print("training MLP...")
#     XTrain, XValid, XTest, YTrain, YValid, YTest = data
    n_samples, n_inputs = XTrain.shape
    batchsize = min(batchsize, n_samples)
    ops.reset_default_graph()


    # placeholders
    X = tf.placeholder("float", [None, n_inputs])
    Y = tf.placeholder("float", [None])

    # forward pass
    prediction = define_forward_pass(X, init_parameters, n_inputs, HL1N, HL2N, n_layers=n_layers)

    # defining a RMSE objective function
    loss = tf.reduce_mean(tf.pow(prediction - Y, 2) )
    optimiser = tf.train.AdamOptimizer(learning_rate).minimize(loss)

    # define minibatch boundaries
    batch_boundaries = list(zip(range(0, n_samples, batchsize), \
                range(batchsize, n_samples, batchsize)))
    if n_samples % batchsize:
        batch_boundaries += [(batch_boundaries[-1][1],n_samples)]
    if len(batch_boundaries) == 0:
        batch_boundaries += [(0,n_samples)]


    RMSE_train, RMSE_valid, RMSE_test = [], [], []
    pred_test_store = []

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(n_iterations):

            for start, end in batch_boundaries:
                #feed in training data minibatch-wise
                sess.run(optimiser, feed_dict = {X: XTrain[start:end], \
                                                Y: YTrain[start:end]})

            pred_train = sess.run(prediction, feed_dict={X: XTrain, Y: YTrain})
            pred_test = sess.run(prediction, feed_dict={X: XTest, Y: YTest})
            pred_test_store.append(pred_test)

            diff_train = YTrain - pred_train
            RMSE_train.append( np.sqrt(np.mean(np.square(diff_train ) ) ) )

            diff_test = YTest - pred_test
            RMSE_test.append( np.sqrt(np.mean(np.square(diff_test ) ) ) )

    if forest is None:  # vanilla neural net
        return RMSE_test[amin], pred_test_store[amin]
    else:               # RF-initialised neural net
        RF_predictions_test = forest.predict(XTest)
        RF_score_test = np.sqrt( np.mean (np.square(RF_predictions_test-np.squeeze(YTest) ) )  )
    return RF_score_test, RF_predictions_test


In [14]:
data = pd.read_csv(r"....\wdbc.data", header = None)

In [15]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [16]:
cols = ['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean', 'concave_points_mean', 'symmetry_mean', 'fractal_dimension_mean', 'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se', 'concavity_se', 'concave_points_se', 'symmetry_se', 'fractal_dimension_se', 'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst', 'smoothness_worst', 'compactness_worst', 'concavity_worst', 'concave', 'points_worst_symmetry_worst', 'fractal_dimension_worst']

In [17]:
data.columns = cols

In [18]:
data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave,points_worst_symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [19]:
data.drop('id',axis=1,inplace=True)

In [20]:
data['diagnosis'] = data['diagnosis'].map({'M':1,'B':0})

In [21]:
datas = pd.DataFrame(preprocessing.scale(data.iloc[:,1:32]))
datas.columns = list(data.iloc[:,1:32].columns)
datas['diagnosis'] = data['diagnosis']

In [28]:
data.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave,points_worst_symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [32]:
predictors = data.columns[1:31]
target = "diagnosis"

X = data.loc[:,predictors]
y = np.ravel(data.loc[:,[target]])

# Split the dataset in train and test:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
print ('Shape of training set : %i || Shape of test set : %i' % (x_train.shape[0],x_test.shape[0]) )
print ('The dataset is very small so simple cross-validation approach should work here')
print ('There are very few data points so 10-fold cross validation should give us a better estimate')

Shape of training set : 426 || Shape of test set : 143
The dataset is very small so simple cross-validation approach should work here
There are very few data points so 10-fold cross validation should give us a better estimate


In [78]:
for i in tqdm(range(ntrees)):
    errors, preds = [], []
    print ("Training network for tree", i+1, "of", ntrees )
    
    rf = RandomForestClassifier(n_estimators=15)
    rf = rf.fit(x_train, y_train)
    predicted = rf.predict(x_test)
    predictions = []
    acc_test = metrics.accuracy_score(y_test, predicted)
    print ('The accuracy on test data is %s' % (round(acc_test,2)))
    init_parameters = get_network_initialisation_parameters(rf)
    RMSE, pred = run_neural_net(x_train, x_test, y_train, y_test, init_parameters,
                             forest=rf, keep_sparse=True)

    errors += [RMSE]
    predictions += [np.atleast_2d(np.ravel(pred))]

    # average the (scalar) predictions across all trees
    all_preds = np.concatenate(predictions, axis=0)
    avg_pred = np.mean( all_preds, axis=0)

    # compute RMSE
    RMSE = np.sqrt( np.mean (np.square(avg_pred - np.squeeze(y_test)) )  )
    print ("score for averaged prediction of individual networks:", RMSE )

  0%|                                                                                           | 0/15 [00:00<?, ?it/s]

Training network for tree 1 of 15
The accuracy on test data is 0.96
training MLP...


  7%|█████▌                                                                             | 1/15 [00:02<00:31,  2.25s/it]

score for averaged prediction of individual networks: 0.20483662259967567
Training network for tree 2 of 15
The accuracy on test data is 0.97
training MLP...


 13%|███████████                                                                        | 2/15 [00:04<00:28,  2.20s/it]

...

score for averaged prediction of individual networks: 0.25087260300212727
Training network for tree 14 of 15
The accuracy on test data is 0.96
training MLP...


 93%|████████████████████████████████████████████████████████████████████████████▌     | 14/15 [00:31<00:02,  2.18s/it]

score for averaged prediction of individual networks: 0.20483662259967567
Training network for tree 15 of 15
The accuracy on test data is 0.95
training MLP...


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:33<00:00,  2.25s/it]

score for averaged prediction of individual networks: 0.2212488394343549
